# Test Model

In [1]:
import os
from pathlib import Path
import sys
sys.path.append(str(Path(os.getcwd()).parent))

from settings.global_settings import GlobalSettings

config = GlobalSettings.get_config(
    config_file = "../config.ini",
    secrets_file = "../secrets.ini"
)
from dataset.video_loader import VideoDataLoader
from dataset.video_dataset import VideoDataset
from model.training_loop import train
from torch.utils.data import random_split
from model.multimodal_har_model import MultiModalHARModel

2025-10-20 22:18:50,913 - INFO - Sentry DSN set to: https://f4f21cc936b3ba9f5dbc1464b7a40ea4@o4504168838070272.ingest.us.sentry.io/4506464560414720


Loading config...
Loading secrets...


2025-10-20 22:18:50,914 - INFO - Sentry initialized with environment: development


## Initializing Training

**Creating Dataloaders**

In [2]:
video_data_loader = VideoDataLoader(
    path=config.model_settings.video_data_dir
)

video_dataset = VideoDataset(
    video_data_loader=video_data_loader,
    normalization_type="across_frames",
)

len(video_dataset)
for _ in video_dataset:
    pass
len(video_dataset.labels_map)

2025-10-20 22:18:53,220 - INFO - [VideoDataLoader] Loding action videos for action: a01
2025-10-20 22:18:53,604 - INFO - [VideoDataLoader] Loding action videos for action: a02
2025-10-20 22:18:54,031 - INFO - [VideoDataLoader] Loding action videos for action: a03
2025-10-20 22:18:54,381 - INFO - [VideoDataLoader] Loding action videos for action: a04
2025-10-20 22:18:54,892 - INFO - [VideoDataLoader] Loding action videos for action: a05
2025-10-20 22:18:55,212 - INFO - [VideoDataLoader] Loding action videos for action: a06
2025-10-20 22:18:55,631 - INFO - [VideoDataLoader] Loding action videos for action: a08
2025-10-20 22:18:56,358 - INFO - [VideoDataLoader] Loding action videos for action: a09
2025-10-20 22:18:57,235 - INFO - [VideoDataLoader] Loding action videos for action: a11
2025-10-20 22:18:57,520 - INFO - [VideoDataLoader] Loding action videos for action: a12


10

**Splitting Train and Test Datasets**

In [3]:
num_total = len(video_dataset)
num_train = int(0.8 * num_total)
num_test = num_total - num_train
train_dataset, test_dataset = random_split(video_dataset, [num_train, num_test])

**Creating Model**

In [ ]:

har_model = MultiModalHARModel(
    obj_in=train_dataset[0].graphs_objects[0].x.shape[1],
    joint_in=train_dataset[0].graphs_joints[0].x.shape[1],
    gat_hidden=128,
    gat_out=128,
    temporal_hidden=128,
    num_classes=len(video_dataset.labels_map), 
    dropout=0.5,
)

## Training Model

In [ ]:
train_history = train(
    model=har_model,
    video_dataset=train_dataset,
    device='mps',
    epochs=25,
    weight_decay=1e-4,
)

2025-10-20 22:19:03,031 - INFO - Starting training loop...
Epoch 1/25: 100%|██████████| 1180/1180 [06:38<00:00,  2.96it/s]
2025-10-20 22:25:41,573 - INFO - Epoch 1/25, Loss: 2.0608
Epoch 2/25: 100%|██████████| 1180/1180 [06:17<00:00,  3.13it/s]
2025-10-20 22:31:58,635 - INFO - Epoch 2/25, Loss: 1.4806
Epoch 3/25: 100%|██████████| 1180/1180 [06:22<00:00,  3.09it/s]
2025-10-20 22:38:21,047 - INFO - Epoch 3/25, Loss: 1.2626
Epoch 4/25: 100%|██████████| 1180/1180 [06:18<00:00,  3.12it/s]
2025-10-20 22:44:39,729 - INFO - Epoch 4/25, Loss: 1.1304
Epoch 5/25: 100%|██████████| 1180/1180 [21:10<00:00,  1.08s/it]  
2025-10-20 23:05:50,509 - INFO - Epoch 5/25, Loss: 1.0244
Epoch 6/25: 100%|██████████| 1180/1180 [06:00<00:00,  3.27it/s]
2025-10-20 23:11:50,956 - INFO - Epoch 6/25, Loss: 0.9403
Epoch 7/25: 100%|██████████| 1180/1180 [06:00<00:00,  3.27it/s]
2025-10-20 23:17:51,370 - INFO - Epoch 7/25, Loss: 0.8586
Epoch 8/25: 100%|██████████| 1180/1180 [05:51<00:00,  3.35it/s]
2025-10-20 23:23:43,2

**Saving Model**

In [6]:
har_model.save(
    training_history=train_history
)

2025-10-21 08:13:52,393 - INFO - Saving model to /Volumes/KODAK/masters/model/validation_datasets/NW-UCLA/model/har_model_v0.0.0_nw_ucla_2025-10-21 08:13:52.392482.pht...
2025-10-21 08:13:52,576 - INFO - Model saved successfully.


In [7]:
test_video_indices = test_dataset.indices
print("Test video IDs:", test_video_indices)

Test video IDs: [64, 798, 128, 959, 1158, 1205, 1453, 53, 915, 762, 578, 1130, 1006, 73, 1287, 690, 1171, 810, 27, 748, 809, 183, 407, 444, 540, 976, 919, 1111, 1340, 84, 576, 606, 228, 287, 1013, 336, 431, 1462, 372, 1368, 1378, 202, 884, 855, 509, 1337, 549, 590, 369, 226, 243, 1132, 984, 538, 14, 756, 1243, 113, 317, 735, 531, 613, 16, 787, 541, 956, 903, 1225, 642, 1406, 1230, 667, 358, 664, 1365, 120, 1272, 605, 199, 559, 342, 1066, 1157, 791, 1294, 1041, 246, 302, 661, 722, 927, 295, 1057, 1242, 1219, 0, 633, 515, 871, 713, 691, 1256, 121, 1460, 867, 947, 33, 921, 673, 944, 175, 245, 930, 1367, 676, 624, 1457, 1316, 556, 1304, 318, 693, 1134, 1353, 255, 1226, 1089, 768, 726, 939, 297, 326, 496, 1428, 627, 694, 347, 276, 176, 1321, 1466, 914, 153, 1113, 385, 542, 799, 2, 614, 329, 1103, 835, 331, 880, 1383, 351, 205, 118, 1016, 983, 1330, 675, 934, 696, 507, 1400, 648, 380, 1300, 926, 341, 1357, 343, 1098, 1380, 100, 961, 210, 437, 831, 843, 641, 901, 719, 31, 981, 895, 187, 596, 

In [8]:
import torch
from datetime import datetime

model_settings = config.model_settings
save_dir = "~/masters-models"
os.makedirs(save_dir, exist_ok=True)

save_path = os.path.join(
    save_dir,
    f"har_model_{model_settings.model_version}_{model_settings.dataset_prefix}_100perc_{datetime.now()}.pht"
)
torch.save({
"model_state_dict": har_model.state_dict(),
"training_history": train_history,
"model_config": har_model._model_config,
"test_set_indices": test_video_indices,
}, save_path)


## Running tests

**Accuracy**

In [9]:
def evaluate(model, dataset, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for i in range(len(dataset)):
            sample = dataset[i]
            label = sample.label.to(device)

            # Move all graph tensors to device
            graphs_objects = [g.to(device) for g in sample.graphs_objects]
            graphs_joints = [g.to(device) for g in sample.graphs_joints]

            # Forward pass
            output = model(graphs_objects, graphs_joints)

            # Compute prediction
            if output.dim() == 1:
                predicted = torch.argmax(output).unsqueeze(0)
            else:
                _, predicted = torch.max(output, dim=1)

            correct += (predicted == label).sum().item()
            total += 1

    accuracy = 100 * correct / total
    return accuracy


In [10]:
accuracy_evaluation = evaluate(har_model, test_dataset, "mps")
print(f"Test Accuracy: {accuracy_evaluation:.2f}%")

Test Accuracy: 58.64%
